**Exercise 1**

What is the wild-type production of your group’s compound?

In [1]:
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')
print(model.summary())

envcond = {'EX_glc__D_e': (-15.0, 100000.0),
           'EX_o2_e':(0,1000)}

#EX_mal__L_e L-malate
#EX_mal__D_e D-malate

from mewpy.simulation import get_simulator
simul = get_simulator(model, envcond=envcond)
result = simul.simulate(method='FBA')
print(result)
print('L-Malate wild type flux: ', result.fluxes['EX_mal__L_e'])
print('D-Malate wild type flux: ',result.fluxes['EX_mal__D_e'])

Objective
1.0 BIOMASS_Ec_iML1515_core_75p37M = 0.8769972144269731

Uptake
------
Metabolite      Reaction      Flux  C-Number   C-Flux
     ca2_e      EX_ca2_e  0.004565         0    0.00%
      cl_e       EX_cl_e  0.004565         0    0.00%
 cobalt2_e  EX_cobalt2_e 2.192E-05         0    0.00%
     cu2_e      EX_cu2_e 0.0006218         0    0.00%
     fe2_e      EX_fe2_e   0.01409         0    0.00%
  glc__D_e   EX_glc__D_e        10         6  100.00%
       k_e        EX_k_e    0.1712         0    0.00%
     mg2_e      EX_mg2_e  0.007608         0    0.00%
     mn2_e      EX_mn2_e  0.000606         0    0.00%
    mobd_e     EX_mobd_e 6.139E-06         0    0.00%
     nh4_e      EX_nh4_e     9.471         0    0.00%
     ni2_e      EX_ni2_e 0.0002833         0    0.00%
      o2_e       EX_o2_e     22.13         0    0.00%
      pi_e       EX_pi_e     0.846         0    0.00%
     so4_e      EX_so4_e    0.2208         0    0.00%
     zn2_e      EX_zn2_e 0.0002991         0    0.00%



**Exercício 2**

What are the maximum compound production capabilities?

In [7]:
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')
envcond = {'EX_glc__D_e': (-15.0, 100000.0),
           'EX_o2_e':(0,1000)}

from mewpy.simulation import get_simulator
simul = get_simulator(model, envcond=envcond)

simul.FVA(reactions=['EX_mal__L_e', 'EX_mal__D_e'],format='df')


Objective
1.0 BIOMASS_Ec_iML1515_core_75p37M = 0.8769972144269693

Uptake
------
Metabolite      Reaction      Flux  C-Number   C-Flux
     ca2_e      EX_ca2_e  0.004565         0    0.00%
      cl_e       EX_cl_e  0.004565         0    0.00%
 cobalt2_e  EX_cobalt2_e 2.192E-05         0    0.00%
     cu2_e      EX_cu2_e 0.0006218         0    0.00%
     fe2_e      EX_fe2_e   0.01409         0    0.00%
  glc__D_e   EX_glc__D_e        10         6  100.00%
       k_e        EX_k_e    0.1712         0    0.00%
     mg2_e      EX_mg2_e  0.007608         0    0.00%
     mn2_e      EX_mn2_e  0.000606         0    0.00%
    mobd_e     EX_mobd_e 6.139E-06         0    0.00%
     nh4_e      EX_nh4_e     9.471         0    0.00%
     ni2_e      EX_ni2_e 0.0002833         0    0.00%
      o2_e       EX_o2_e     22.13         0    0.00%
      pi_e       EX_pi_e     0.846         0    0.00%
     so4_e      EX_so4_e    0.2208         0    0.00%
     zn2_e      EX_zn2_e 0.0002991         0    0.00%



,Reaction ID,Minimum,Maximum
0,EX_mal__L_e,0.0,1.750872
1,EX_mal__D_e,0.0,0.000000


**Exercício 3**

Use different optimization objective functions to improve the production of the compound, considering that cells have evolved for maximum growth.

*a)* Evaluate if single gene deletions enhance the production of the compound. Rank the mutants obtained according to the compound production capacity and growth performance.

In [10]:
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml')
envcond = {'EX_glc__D_e': (-15.0, 100000.0),
           'EX_o2_e':(0,1000)}
from mewpy.simulation import get_simulator
simul = get_simulator(model, envcond=envcond)

# Define the target
PRODUCT_ID = 'EX_mal__L_e'
BIOMASS_ID = 'BIOMASS_Ec_iML1515_core_75p37M'

#Optimization objectives
from mewpy.optimization.evaluation import  BPCY, MinCandSize, AggregatedSum, TargetFlux
f1 = BPCY(BIOMASS_ID, PRODUCT_ID, method='pFBA')
f2 = MinCandSize()
fevaluation = AggregatedSum([f1,f2])
fevaluation1 = TargetFlux(PRODUCT_ID)

from mewpy.problems import GKOProblem
problem = GKOProblem(model, fevaluation = [fevaluation, fevaluation1], envcond=envcond)

from mewpy.optimization import EA
ea = EA(problem, max_generations= 100, visualizer=True)
final_pop = ea.run()

from mewpy.utils.constants import ModelConstants
ModelConstants.RESET_SOLVER = True




C:\Anaconda3\envs\py37\lib\site-packages\cobra\util\solver.py:508: UserWarning:

Solver status is 'infeasible'.



KeyboardInterrupt: 